In [ ]:
!pip install torch torchtext torchcrf pytorch-crf

In [1]:
#!pip install datasets==2.0.1
from datasets import load_dataset, load_from_disk

In [ ]:
!pip install neattext

In [ ]:
pip install pytorch-crf

In [2]:
from datasets import load_dataset, load_from_disk


# Later, load the dataset from disk
dataset = load_dataset("wikiann", "tr")

# Verify that the dataset is loaded correctly
print(dataset)


DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 20000
    })
})


In [3]:
import pandas as pd

In [4]:
print(dataset["train"][0])

{'tokens': ['3.lük', 'maçında', 'Slovenya', 'Millî', 'Basketbol', "Takımı'nı", 'yendikleri', 'maçta', '23', 'sayı', ',', '6', 'ribaund', ',', '2', 'blok', 'istatistikleriyle', 'oynamış', 've', '12', 'faul', 'yaptırmıştır', '.'], 'ner_tags': [0, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'langs': ['tr', 'tr', 'tr', 'tr', 'tr', 'tr', 'tr', 'tr', 'tr', 'tr', 'tr', 'tr', 'tr', 'tr', 'tr', 'tr', 'tr', 'tr', 'tr', 'tr', 'tr', 'tr', 'tr'], 'spans': ["ORG: Slovenya Millî Basketbol Takımı'nı"]}


In [5]:
ner_encoding = {0: "O", 1: "entity", 2: "entity", 3: "entity", 4: "entity", 5: "entity", 6: "entity"}


train_tokens = []
train_tags = []
for sample in dataset["train"]:
  train_tokens.append(' '.join(sample["tokens"]))
  train_tags.append(' '.join([ner_encoding[a] for a in sample["ner_tags"]]))

test_tokens = []
test_tags = []
for sample in dataset["train"]:
  test_tokens.append(' '.join(sample["tokens"]))
  test_tags.append(' '.join([ner_encoding[a] for a in sample["ner_tags"]]))

df_train = pd.DataFrame({"sentence": train_tokens, "tags": train_tags})
df_test = pd.DataFrame({"sentence": test_tokens, "tags": test_tags})

In [6]:
ner_encoding = {0: "O", 1: "entity", 2: "entity", 3: "entity", 4: "entity", 5: "entity", 6: "entity"}

texts = []
for sample in dataset["train"]:
  texts.append(' '.join(sample["tokens"]))
for sample in dataset["test"]:
  texts.append(' '.join(sample["tokens"]))

labels = []
for sample in dataset["train"]:
  labels.append([ner_encoding[a] for a in sample["ner_tags"]])
for sample in dataset["test"]:
  labels.append([ner_encoding[a] for a in sample["ner_tags"]])

In [7]:
pip install TorchCRF

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install torchcrf

Note: you may need to restart the kernel to use updated packages.


In [9]:
from TorchCRF import CRF

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torchcrf import CRF

# Define NERDataset class
class NERDataset(Dataset):
    def __init__(self, texts, labels, vocab, tag_to_idx):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab
        self.tag_to_idx = tag_to_idx

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = [self.vocab.get(word, self.vocab['<UNK>']) for word in self.texts[idx].split()]
        labels = [self.tag_to_idx[tag] for tag in self.labels[idx]]
        return torch.tensor(text, dtype=torch.long), torch.tensor(labels, dtype=torch.long)

# Define BiLSTM_CRF model class
class BiLSTM_CRF(nn.Module):
    def __init__(self, vocab_size, tag_to_idx, embedding_dim=200, hidden_dim=256):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_idx = tag_to_idx
        self.tagset_size = len(tag_to_idx)

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=2, bidirectional=True, dropout=0.5)
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        self.crf = CRF(self.tagset_size, batch_first=True)

    def forward(self, sentence, tags=None):
        embedded = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embedded)
        tag_scores = self.hidden2tag(lstm_out)

        if tags is not None:
            mask = (sentence != 0).bool()
            loss = -self.crf(tag_scores, tags, mask=mask)
            return loss
        else:
            best_tags = self.crf.decode(tag_scores)
            return best_tags

# Prepare data
def prepare_data(texts, labels):
    vocab = {'<PAD>': 0, '<UNK>': 1}
    tag_to_idx = {'O': 0, 'entity': 1}

    for text in texts:
        for word in text.split():
            if word not in vocab:
                vocab[word] = len(vocab)

    return vocab, tag_to_idx

# Collate function for DataLoader
def collate_fn(batch):
    texts, labels = zip(*batch)
    max_len = max(len(x) for x in texts)
    padded_texts = [torch.cat([x, torch.zeros(max_len - len(x), dtype=torch.long)]) for x in texts]
    padded_labels = [torch.cat([y, torch.zeros(max_len - len(y), dtype=torch.long)]) for y in labels]
    return torch.stack(padded_texts), torch.stack(padded_labels)

# Example data (texts and labels should be extended with more diverse data)


# Check and align lengths of texts and labels
for i in range(len(texts)):
    text_length = len(texts[i].split())
    label_length = len(labels[i])
    if text_length != label_length:
        raise ValueError(f"Text and label lengths do not match at index {i}: {text_length} != {label_length}")

# Prepare vocabulary and tag_to_idx
vocab, tag_to_idx = prepare_data(texts, labels)
dataset = NERDataset(texts, labels, vocab, tag_to_idx)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

# Initialize model, optimizer, and device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM_CRF(len(vocab), tag_to_idx, embedding_dim=400, hidden_dim=512).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)  # Adjusted learning rate

# Training loop
num_epochs = 3000
for epoch in range(num_epochs):
    total_loss = 0
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        loss = model(inputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}")

# Test sentence
test_sentence = "benim şirketim kazanacaktır, onun adı da turkcell"
test_tensor = torch.tensor([vocab.get(word, vocab['<UNK>']) for word in test_sentence.split()], dtype=torch.long).to(device)
with torch.no_grad():
    best_tags = model(test_tensor.unsqueeze(0))[0]

idx_to_tag = {i: tag for tag, i in tag_to_idx.items()}
predicted_labels = [idx_to_tag[i] for i in best_tags]
print("Test cümlesi:", test_sentence)
print("Tahmin edilen etiketler:", predicted_labels)

KeyboardInterrupt: 

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torchcrf import CRF

# Define NERDataset class
class NERDataset(Dataset):
    def __init__(self, texts, labels, vocab, tag_to_idx):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab
        self.tag_to_idx = tag_to_idx

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = [self.vocab.get(word, self.vocab['<UNK>']) for word in self.texts[idx].split()]
        labels = [self.tag_to_idx[tag] for tag in self.labels[idx]]
        return torch.tensor(text, dtype=torch.long), torch.tensor(labels, dtype=torch.long)

# Define BiLSTM_CRF model class
class BiLSTM_CRF(nn.Module):
    def __init__(self, vocab_size, tag_to_idx, embedding_dim=200, hidden_dim=256):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_idx = tag_to_idx
        self.tagset_size = len(tag_to_idx)

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=2, bidirectional=True, dropout=0.5)
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        self.crf = CRF(self.tagset_size, batch_first=True)

    def forward(self, sentence, tags=None):
        embedded = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embedded)
        tag_scores = self.hidden2tag(lstm_out)

        if tags is not None:
            mask = (sentence != 0).bool()
            loss = -self.crf(tag_scores, tags, mask=mask)
            return loss
        else:
            best_tags = self.crf.decode(tag_scores)
            return best_tags

# Prepare data
def prepare_data(texts, labels):
    vocab = {'<PAD>': 0, '<UNK>': 1}
    tag_to_idx = {'O': 0, 'entity': 1}

    for text in texts:
        for word in text.split():
            if word not in vocab:
                vocab[word] = len(vocab)

    return vocab, tag_to_idx

# Collate function for DataLoader
def collate_fn(batch):
    texts, labels = zip(*batch)
    max_len = max(len(x) for x in texts)
    padded_texts = [torch.cat([x, torch.zeros(max_len - len(x), dtype=torch.long)]) for x in texts]
    padded_labels = [torch.cat([y, torch.zeros(max_len - len(y), dtype=torch.long)]) for y in labels]
    return torch.stack(padded_texts), torch.stack(padded_labels)

# Example data (texts and labels should be extended with more diverse data)


# Check and align lengths of texts and labels
for i in range(len(texts)):
    text_length = len(texts[i].split())
    label_length = len(labels[i])
    if text_length != label_length:
        raise ValueError(f"Text and label lengths do not match at index {i}: {text_length} != {label_length}")

# Prepare vocabulary and tag_to_idx
vocab, tag_to_idx = prepare_data(texts, labels)
dataset = NERDataset(texts, labels, vocab, tag_to_idx)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

# Initialize model, optimizer, and device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM_CRF(len(vocab), tag_to_idx, embedding_dim=800, hidden_dim=1024).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)  # Adjusted learning rate

# Training loop
num_epochs = 3000
for epoch in range(num_epochs):
    total_loss = 0
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        loss = model(inputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}")

# Test sentence
test_sentence = "benim şirketim kazanacaktır, onun adı da turkcell"
test_tensor = torch.tensor([vocab.get(word, vocab['<UNK>']) for word in test_sentence.split()], dtype=torch.long).to(device)
with torch.no_grad():
    best_tags = model(test_tensor.unsqueeze(0))[0]

idx_to_tag = {i: tag for tag, i in tag_to_idx.items()}
predicted_labels = [idx_to_tag[i] for i in best_tags]
print("Test cümlesi:", test_sentence)
print("Tahmin edilen etiketler:", predicted_labels)

In [13]:
test_sentence = "Vatan Bilgisayardan cok iyi bir Türk Telekomdan daha iyi"
test_tensor = torch.tensor([vocab.get(word, vocab['<UNK>']) for word in test_sentence.split()], dtype=torch.long).to(device)
with torch.no_grad():
    best_tags = model(test_tensor.unsqueeze(0))[0]

idx_to_tag = {i: tag for tag, i in tag_to_idx.items()}
predicted_labels = [idx_to_tag[i] for i in best_tags]
print("Test cümlesi:", test_sentence)
print("Tahmin edilen etiketler:", predicted_labels)

Test cümlesi: Vatan Bilgisayardan cok iyi bir Türk Telekomdan daha iyi
Tahmin edilen etiketler: ['entity', 'entity', 'O', 'O', 'O', 'entity', 'entity', 'O', 'O']


In [14]:
test_sentence = "bugün mediamarkt mağazasından iphone 7 plus satın aldım ama sorunlu çıktı"
test_tensor = torch.tensor([vocab.get(word, vocab['<UNK>']) for word in test_sentence.split()], dtype=torch.long).to(device)
with torch.no_grad():
    best_tags = model(test_tensor.unsqueeze(0))[0]

idx_to_tag = {i: tag for tag, i in tag_to_idx.items()}
predicted_labels = [idx_to_tag[i] for i in best_tags]
print("Test cümlesi:", test_sentence)
print("Tahmin edilen etiketler:", predicted_labels)

Test cümlesi: bugün mediamarkt mağazasından iphone 7 plus satın aldım ama sorunlu çıktı
Tahmin edilen etiketler: ['entity', 'entity', 'entity', 'entity', 'O', 'entity', 'entity', 'entity', 'O', 'O', 'O']


In [18]:
import pickle
#pickle.dump(model,open("entity_cikarici.ai2","wb"))

In [19]:
model2 = pickle.load(open("entity_cikarici.ai","rb"))

/home/pc1/venv/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [20]:
test_sentence = "bugün mediamarkt mağazasından iphone 7 plus satın aldım ama sorunlu çıktı"
test_tensor = torch.tensor([vocab.get(word, vocab['<UNK>']) for word in test_sentence.split()], dtype=torch.long).to(device)
with torch.no_grad():
    best_tags = model2(test_tensor.unsqueeze(0))[0]

idx_to_tag = {i: tag for tag, i in tag_to_idx.items()}
predicted_labels = [idx_to_tag[i] for i in best_tags]
print("Test cümlesi:", test_sentence)
print("Tahmin edilen etiketler:", predicted_labels)

Test cümlesi: bugün mediamarkt mağazasından iphone 7 plus satın aldım ama sorunlu çıktı
Tahmin edilen etiketler: ['O', 'entity', 'entity', 'entity', 'entity', 'entity', 'O', 'entity', 'O', 'O', 'O']


/home/pc1/venv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:917: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:1424.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


In [21]:
!pip install zemberek-python;